<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/Pending/330Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 写真の減色

カラー画像データは1600万色程度($=256^3$)の色彩を表現できるが、これは一般的な写真の画素数よりも多い。つまり、全く使っていない色はたくさんある。また、次の写真のように、カラー写真とはいっても、実際に使っている色はそれほど多くないように見えるものもある。

![](https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg)

(CC BY 2.0 2009 SteFou! via Flickr)

この写真を、できるだけ少ない色数で表示してみよう。

## 1.白黒の場合

まず、この画像を入手し、色彩を落として、numpy arrayの形にする。414 x 640W pixelの画像なので、サイズ(414,640)の実数のarrayの各要素がそれぞれの画素の明るさを表す。

> imageioライブラリのimread関数を用いることで、URLから画像データをarrayとしてとりいれる。

In [ ]:
from imageio import imread

img = imread("https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg")
img.shape

画像の画素ごとのデータは、通常はR(赤),G(緑),B(青)それぞれ0〜255の256段階で表現される。3色を平均して白黒画像にする。

> imgのデータ形式はnumpyのarray(配列)形式になっているが、あとでいろいろデータ加工をしやすいように、numpyも読みこんでおく。

In [ ]:
# RGB方向の平均をとり、255で割る。
import numpy as np

#平均値は実数になってしまうので、8ビット負号なし整数(0〜255)に変換しておく。
gray = np.average(img, axis=2).astype(np.uint8)
gray.shape

In [ ]:
# grayの中身は二次元配列で、ピクセルの明るさを表す0〜255の整数が格納されている。
gray

In [ ]:
# ただの配列なので、座標を指定して数値をとりだすこともできる。
gray[0,0]

In [ ]:
# PILは画像を扱うライブラリ。
import PIL

# 配列を、画像データに変換する。
image = PIL.Image.fromarray(gray)
image

In [ ]:
# 画像はただの数値配列なので、部分を切りだしたり加工したりするのも簡単。
# 以下では、写真の左上すみから(100,100)を始点として、(150,150)ピクセル分を切り出す。

part = gray[100:250, 100:250]
PIL.Image.fromarray(part)

ヒストグラムを作る。

In [ ]:
# numpyのヒストグラム関数は、配列grayの最小値と最大値の間を8等分してデータ個数を数える。
# binには9個の値が入っていることに注意。
freq, bin = np.histogram(gray, bins=8)
freq, bin

In [ ]:
# matplotlibを用いてプロットしてみる。
from matplotlib import pyplot as plt

plt.bar(bin[:-1], freq, width=255/9)

ピクセルの明るさが、100〜220に集中しているのがわかる。

### 1.1 単純に階調を8等分して、8階調にする

何も考えず、この画像を8階調に落す。手っ取り早い方法は、0〜255の階調を32で割り、それを再度32倍するという方法である。演算子`//`を使って整数を整数で割ると、小数以下が切りすてられる。

In [ ]:
# //は整数同士の割り算です。商も整数です。
gray8 = (gray // 32) * 32
PIL.Image.fromarray(gray8)

なんかのっぺりとしてわかりにくい絵になってしまった。コントラストが小さいせいだろう。灰色の階調をもう少しこまかくとりたい。

### 1.2 ヒストグラムにもとづいた彩色



ヒストグラムにより、暗い色が少なく、明るい色に偏っていることがわかっているので、明るいところを、より細かい階調に分類したほうがよさそうだ。

画像の統計的性質をはっきりさせるために、ピクセルを暗い順にソートしてみる。



In [ ]:
# まず、2次元のピクセルを1次元にします。
flatten = gray.reshape(-1)
flatten

In [ ]:
# それをsortします。
tone = np.sort(flatten)
tone

In [ ]:
# プロットしてみましょう。
plt.plot(tone)
plt.xlabel("pixels")
plt.ylabel("brightness")

上のグラフの横軸は、写真のピクセル数に対応している。

8段階の明暗を使うものとします。できるだけ、まんべんなくいろんな段階のピクセルを使いたいなら、上のグラフの横軸を8等分するようにするのが良いだろう。

こんな感じ。

In [ ]:
# 全ピクセル数の1/8は?
pixels = len(tone) // 8
pixels

In [ ]:
# Xは、pixels間隔の8点
X = [pixels*i for i in range(9)]

# Yは、Xと同じ形の配列で、値は255
Y = np.zeros_like(X) + 255

# stemプロットは垂直線を描く。
plt.stem(X,Y)

In [ ]:
# 上のグラフと重ねる。
plt.plot(tone)
plt.xlabel("pixels")
plt.ylabel("brightness")
plt.stem(X,Y)

明るさが0〜108なら、一番暗いピクセルにし、208以上なら一番明るいピクセルにすればいい、ということが予想できる。これを実際にやってみよう。

とりあえず、8つのグループの境目を特定する。


In [ ]:
for i in range(8):
    x = i*pixels
    print(i, tone[x])

明るさが0〜108の点は、明るさ$(0+108)\div 2=54$に置きかえればいいようだ。

grayのピクセルを、範囲ごとに置きかえていく。

1ピクセルずつ、条件分けして色をおきかえる。

In [ ]:
graded = np.zeros_like(gray)
for y in range(gray.shape[0]):
    for x in range(gray.shape[1]):
        if 0 <= gray[y,x] < 108:
            graded[y,x] = 54
        elif gray[y,x] < 128:
            graded[y,x] = (128+108)//2
        elif gray[y,x] < 141:
            graded[y,x] = (141+128)//2
        elif gray[y,x] < 154:
            graded[y,x] = (154+141)//2
        elif gray[y,x] < 170:
            graded[y,x] = (170+154)//2
        elif gray[y,x] < 186:
            graded[y,x] = (186+170)//2
        elif gray[y,x] < 208:
            graded[y,x] = (208+186)//2
        else:
            graded[y,x] = (255+208)//2

PIL.Image.fromarray(graded)        

ちょっとメリハリがついて、背景と桜が見分けられるようになった。

numpyのfancy indexを使うと、もっと簡単かつ高速に色の置き換えができる。


In [ ]:
graded = np.zeros_like(gray)
graded[gray < 108] = 54
graded[(108 <= gray) & (gray < 128)] = (108+128)//2
graded[(128 <= gray) & (gray < 141)] = (128+141)//2
graded[(141 <= gray) & (gray < 154)] = (141+154)//2
graded[(154 <= gray) & (gray < 170)] = (154+170)//2
graded[(170 <= gray) & (gray < 186)] = (170+186)//2
graded[(186 <= gray) & (gray < 208)] = (186+208)//2
graded[(208 <= gray) & (gray < 255)] = (208+255)//2


PIL.Image.fromarray(graded)        

## 2. カラーの場合

デジタルカラー画像は赤、緑、青それぞれの強度が256段階あり、1600万通りの色彩がありうる。RGBをそれぞれ8段階にしたとしても、64色は必要になる。8色にまで落とすためには、3色をon/offの2段階にまで落とす必要があり、やる前からうまくいかないのは目に見えている。

In [ ]:
# imgデータを壊してしまわないように、コピーを作る。
simple = img.copy()

# simpleは3次元の配列
simple.shape

In [ ]:
# 画素の輝度が128より小さい点はすべて0にする。
simple[img<128]=0

# 128より大きい点はすべて255にする。
simple[img>=128]=255

display(PIL.Image.fromarray(simple))

酷い。



もとの絵に含まれているピクセルの色の分布を、RGB3次元の空間で表してみる。

In [ ]:
# すべてのピクセルをプロットするのは大変なので、縦横それぞれ1/4にした小さい画像を作る。
tiny = img[::4, ::4]
PIL.Image.fromarray(tiny)

In [ ]:
# 写真に使われていた色を、RGB空間にプロットする。
# plotlyを用いると、三次元プロットをその場で回しながら見ることができる。
# plotlyの使い方はここでは解説しない。

import plotly.graph_objs as go

height, width = tiny.shape[:2]
Npix = height*width

# 3次元配列を2次元に変換する。各行が、RGBの3原色の強度を表す。
pixels = tiny.reshape(Npix, 3)

colors = ['rgb({0},{1},{2})'.format(r,g,b) for r,g,b in pixels[:]]
trace=dict(type='scatter3d',
           x= pixels[:,0],
           y= pixels[:,1],
           z= pixels[:,2],
           mode='markers',
           marker=dict(color=colors,
                       size=3)
          )
fig = go.Figure(data=trace)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'))
fig.show()

使われている色はとても偏っていることがわかる。

そこで、k-平均分類という機械学習の手法を使ってみる。k-平均分類は、機械学習において、多次元空間に散在する多数の点を、近いもの同士で集めて、クラスターにする、「教師なし学習」の一手法である。

桜の写真の画素は、R,G,Bを3つの軸とする立方体の中の点で表される。上の写真には640x414=265000点の画素があり、それらは立方体の中で偏って分布している。おそらくピンクに相当する部分に多数の点が集中し、ほかに緑や黒に相当する領域に小さな集団を作っているはずだ。

そこで、この3次元空間内の点を、近いものどうしをつないでいくことで、領域分割する手法がk-平均法 (k-means classifier)である。

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

2次元でのk-近傍分類器はこんな感じ。10グループに分けろ、と指定すると、10種類に分けてくれる。それぞれのクラスターの重心点で色を代表させれば、色数を減らせる。

機械学習ライブラリscikit-learnに含まれているKMeansを使う。

In [ ]:
# Pythonデータサイエンスハンドブック 5.11.1
from sklearn.cluster import KMeans

height, width = img.shape[:2]
Npix = height*width

#画像をピクセル列に変換する(そうしないとk-meansが使えない)
pixels = img.reshape(Npix, 3)

# 8つの代表的な色をさがさせる = 点を、3次元空間の8つの代表点に結びつける。

# k-meansの初期化。8グループに分ける。
kmeans = KMeans(n_clusters=8, max_iter=2000)

# データから学習
kmeans.fit(pixels)

# 学習結果。8つの中心の座標
kmeans.cluster_centers_

8つの中心の位置=代表色をならべて表示する。

In [ ]:
import numpy as np

# 100x100ピクセルのカラー画像を準備する。
square = np.zeros([200,200,3], dtype=np.uint8)

# 帯状に、8色で塗りわける。
for i in range(8):
    x = i*25
    square[:,x:x+25] = kmeans.cluster_centers_[i]

# 表示
display(PIL.Image.fromarray(square))

In [ ]:
# predict関数は、与えられた点を8つのグループのいずれかに分類してくれる。
groups = kmeans.predict(pixels)
groups

groupsは、pixelsの各ピクセルがどのグループに属するのかを示している。

numpyのfancy indexを用いて、グループの並びを代表色の並びに変換する。

In [ ]:
new_pixels = kmeans.cluster_centers_[groups]
new_pixels

In [ ]:
# 2次元を3次元に戻す。
graded = new_pixels.reshape(height, width, 3)

# 実数を整数(符号なし8ビット、画像データの典型的な数値形式)に変換
graded = graded.astype(np.uint8)


display(PIL.Image.fromarray(graded))


たった8色でもここまで雰囲気が出せた。

どのようにクラスター化されたかを、RGB3次元で見てみよう。

In [ ]:
# 各ピクセルの色が、どの色に減色されたかを散布図で示す。8つの領域に分割されている。

import plotly.graph_objs as go

tiny = img[::4, ::4]
height, width = tiny.shape[:2]
Npix = height*width

# 2次元のピクセル列に
pixels = tiny.reshape(Npix, 3)
# k-meansによる減色
predicted   = kmeans.cluster_centers_[kmeans.predict(pixels)].reshape(Npix, 3)

# 点の座標には原画のピクセル値を用い、彩色には減色したあとの色を指定する。
colors = ['rgb({0},{1},{2})'.format(r,g,b) for r,g,b in predicted[:]]
trace=dict(type='scatter3d',
           x= pixels[:,0],
           y= pixels[:,1],
           z= pixels[:,2],
           mode='markers',
           marker=dict(color=colors,
                       size=3)
          )
fig = go.Figure(data=trace)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'))
fig.show()

もとの写真の色がかなりかたよっていたので、クラスター化の結果は、RGB空間の対角線方向に8等分する形になったことがわかる。

8色が最適とは限らない。色数を増やせば、原画に限りなく近付けることはできるが、何色をつかえば十分良いと言えるだろう。

ここでは、原画と、減色後の画像を比較し、その差を平均二乗誤差で評価することにする。


In [ ]:
np.var(pixels - predicted)

In [ ]:
for ncolors in range(4,16):
    kmeans = KMeans(n_clusters=ncolors, max_iter=2000)
    # データから学習
    kmeans.fit(pixels)
    groups = kmeans.predict(pixels)
    predicted = kmeans.cluster_centers_[groups]
    print(ncolors, np.var(pixels - pred))

色数を増やすと、誤差が減る。この傾向をプロットしてみよう。


In [ ]:
X = []
Y = []
for ncolors in range(3,16):
    kmeans = KMeans(n_clusters=ncolors, max_iter=2000)
    # データから学習
    kmeans.fit(pixels)
    groups = kmeans.predict(pixels)
    predicted = kmeans.cluster_centers_[groups]
    print(ncolors, np.var(pixels - predicted))
    X.append(ncolors)
    Y.append(np.var(pixels - predicted))

plt.yscale('log')
plt.plot(X,Y)
plt.xlabel("Number of centers in k-means")
plt.ylabel("Mean squared error")

あまり特徴がないが、8色よりも色数を増やしても劇的に誤差が減る傾向もないので、8色で十分ではないか。

## 課題

インターネット上で画像を見付け、モノクロとカラーで減色を行う。
* 最適な階調数は8階調とは限らない。ヒストグラムをみたり、誤差を評価するなどして、適切な階調数を選ぶこと。

